An `Interchange` object provides several methods to produce input data for other software. Note that none of these methods aim to export _all_ stored information. INstead, they support a design where the principle source of truth is the rich chemical information in the Interchange object and each export method provides only what the file or object needs.

For starters, let's create a single-molecule `Interchange` just like we did in the `Construction.ipynb` notebook.

In [ ]:
from openff.toolkit import ForceField, Molecule

from openff.interchange import Interchange

molecule = Molecule.from_smiles("c1ccccc1-c2ccccc2")
molecule.generate_conformers(n_conformers=1)

sage = ForceField("openff-2.0.0.offxml")

interchange = Interchange.from_smirnoff(sage, molecule.to_topology())


The simplest example is exported to a PDB file using `Interchange.to_pdb()`

In [ ]:
interchange.to_pdb("biphenyl.pdb")

From here, you can spot-check the result with NGLView or your favorite visualization software.

In [ ]:
import nglview

nglview.show_file("biphenyl.pdb")

GRAMACS coordinate (`.gro`) and topology (`.top`) files can be produced with `Interchange.to_gro()` and `Interchange.to_top()`.

In [ ]:
interchange.to_gro("biphenyl.gro")
interchange.to_top("biphenyl.top")

In [ ]:
nglview.show_file("biphenyl.gro")

An Openmm `System` object can be `Interchange.to_openmm()`. By default, for most use cases, the returned `System` will have its non-bonded interactions split out into several forces.

In [ ]:
openmm_system = interchange.to_openmm()

for force in openmm_system.getForces():
    print(type(force))

This behavior can be toggled off with the argument `combine_nonbonded_forces=True`, which attempts to combine all non-bonded interactions into a single `openmm.NonbondedForce`.

In [ ]:
openmm_system = interchange.to_openmm(combine_nonbonded_forces=True)

for force in openmm_system.getForces():
    print(type(force))

This is effectively what happens internally when `ForceField.create_openmm_system` is called with OpenFF Toolkit version 0.11.0 and newer; internally an `Interchange` object is created and immediately exported to `openmm.System`, which is then returned.

Amber coordinate (`.inpcrd`) and parameter-topology (`.prmtop`) files can be produced with `Interchange.to_inpcrd()` and `Interchange.to_prmtop()`. Note that Amber exports are comparatively less validated than OpenMM and GROMACS exports - please make an effort to validate the accuracy of these outputs!

In [ ]:
interchange.to_prmtop("biphenyl.prmtop")
interchange.to_inpcrd("biphenyl.inpcrd")

LAMMPS data files (`.data`) can be produced with `Interchange.to_lammps()`. Note that LAMMPS exports have only been vlidated for a few test cases and are even more experimental than Amber exports. Please double-check the contents of these files before using them in production!

In [ ]:
interchange.to_lammps("biphenyl.lmp")

For some array-based or machine learning-adjacent applications, you may want a vectorized representation of the parameters. There is an API to get these representation out of each potential handler, either collapsed to uniqe force field parameters or expanded out to every parameter in the system. In the case of the bond handler, the shape is two columns (force constant `k` and equilibrium bond length `r_eq`). The number of rows is either 3 (the number of unique parameters found in the force field) or 23 (the number of bonds in this biphenyl).

In [ ]:
interchange["Bonds"].get_force_field_parameters()

In [ ]:
interchange["Bonds"].get_system_parameters()

Please reach out of we can help transform the contents of an `Interchange` object to be useful for your tools!